In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect


In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()
inspector = inspect(engine)

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column['name'], column['type'])

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
measurement_df = pd.read_sql("select * from Measurement", conn)
station_df = pd.read_sql("select * from Station", conn)

# Calculate the date 1 year ago from the last data point in the database
max_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()

result = []
for d in max_date:
    result = d
year = int(result.split('-')[0])
month = int(result.split('-')[1])
day = int(result.split('-')[2])

twelve_month_start = dt.date(year,month,day) - dt.timedelta(days=365)
twelve_month_end = dt.date(year,month,day)

# Perform a query to retrieve the data and precipitation scores
sel = [Measurement.date, Measurement.prcp]
measurement_list = session.query(*sel).filter(Measurement.date>=twelve_month_start, Measurement.date<=twelve_month_end).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
measurement_data = pd.DataFrame(measurement_list).dropna()

# Sort the dataframe by date
m_data_sorted = measurement_data.sort_values('date', ascending=True)
m_data_sorted['month_year'] = pd.to_datetime(m_data_sorted['date']).dt.to_period('M')
prcp_data = m_data_sorted.groupby(['month_year']).max().reset_index()
prcp_data

# Use Pandas Plotting with Matplotlib to plot the data
date = prcp_data['month_year'].to_list()
prcp = prcp_data['prcp'].to_list()


prcp_plot = prcp_data.plot(kind="bar", title="Precipitation Scores", figsize=(10,7))
prcp_plot.set_xticklabels(prcp_data['month_year'], rotation=45)
prcp_plot.set_xlabel('Month')


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
m_data_sorted.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?

# Perform a query to retrieve the station and total observations
sel = [Measurement.station, Measurement.tobs]
m_station_list = session.query(*sel).all()

# Save the query results as a Pandas DataFrame
m_station_data = pd.DataFrame(m_station_list).dropna()

# Count the number of stations in dataset.
station_count = m_station_data['station'].nunique()

print(f"There are {station_count} stations available in this data set.")

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.

station_count = m_station_data.groupby(['station']).count()
station_count

# List the stations and the counts in descending order.
station_count_sorted = station_count.sort_values('tobs', ascending=False).rename(columns={'tobs': 'count'}).reset_index()
station_count_sorted

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?

lowest_temp = m_station_data['tobs'].loc[m_station_data['tobs'].idxmin()]
highest_temp = m_station_data['tobs'].loc[m_station_data['tobs'].idxmax()]
average_temp = m_station_data['tobs'].mean()

print(f"The lowest temperature in the dataset is {lowest_temp} degrees.")
print(f"The highest temperature in the dataset is {highest_temp} degrees.")
print(f"The average temperature in the dataset is {round(average_temp,2)} degrees.")

In [ ]:
# Choose the station with the highest number of temperature observations.
most_observations = station_count_sorted['station'][0]
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


most_active_measurement_list = session.query(Measurement.tobs).\
                                filter(Measurement.date>=twelve_month_start, Measurement.date<=twelve_month_end).\
                                filter(Measurement.station == most_observations).all()
most_active_temp = pd.DataFrame(most_active_measurement_list)
most_active_temp.plot.hist(bins=12, title="Precipitation Scores", figsize=(10,7))


In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
trip_start = dt.date(2017,3,28)
trip_end = trip_start + dt.timedelta(days=7)

prev_year_trip_start = trip_start - dt.timedelta(days=365)
prev_year_trip_end = trip_end - dt.timedelta(days=365)

calc_temps(prev_year_trip_start, prev_year_trip_end)

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

min_temp = calc_temps(prev_year_trip_start, prev_year_trip_end)[0][0]
avg_temp = calc_temps(prev_year_trip_start, prev_year_trip_end)[0][1]
max_temp = calc_temps(prev_year_trip_start, prev_year_trip_end)[0][2]

x_pos = np.arange(1)
error =  max_temp - min_temp

plt.figure(figsize=(4,10))
plt.bar(x_pos, avg_temp, yerr=error, align='center', alpha=0.5, ecolor='black', capsize=10)
plt.xticks(x_pos)
plt.ylabel('Temp (F)')
plt.title('Trip Avg Temp')
plt.ylim(0, 100)

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

sel = [Measurement.station, Measurement.prcp]
measurement_rainfall = session.query(*sel).filter(Measurement.date>=prev_year_trip_start, Measurement.date<=prev_year_trip_end).all()

rainfall_df = pd.DataFrame(measurement_rainfall)

rainfall_data = rainfall_df.groupby(['station']).sum()
merged_df = pd.merge(station_df, rainfall_data, on='station', how='inner')
rainfall_sorted = merged_df.sort_values('prcp', ascending=False).drop(columns=['id']).set_index('station')

rainfall_sorted